## Carga inicial de los datos

Cargamos con pandas en memoria nuestro dataset de libros.

In [173]:
import pandas as pd

df = pd.read_csv("books.csv")

## Preparación de los datos relevantes para BigML
Para ello vamos a seleccionar únicamente la información que consideramos relevante para BigML a la hora de hacer un modelo de predicción que nos indique cuando un libro va a tener buena acogida por parte de los lectores.

En concreto, nos interesa conocer a cerca de cada libro:
 + El título.
 + Los autores que lo han escrito.
 + La editorial.
 + Valoración media.
 + Número de valoraciones.
 + Fecha de publicación.
 + Generos a los que pertenece
 + Cuántas personas lo han leído.
 + Cuántas personas lo están leyendo.
 + Cuántas personas lo quieren leer.

## Salvamos el resultado en un csv específico
Debido a que los campos "_genres_" y "_authors_" tienen valores multiples separados por comas, guardamos el CSV _"books-bigml.csv"_ con un nuevo separador (**|**).

Además calculamos una nueva columna *"good_book"* en la que indicamos si un libro es bueno comprobando si valoración media es al menos como la de la media de la de todos los libros y además lo hayan valorado al menos 50 individuos. 

In [175]:
books_average_mean = df.average_rating.mean()

def isGoodBook(book):
    return book.average_rating > books_average_mean

df = df.assign(good_book=isGoodBook)
df_bigml = df[["title", "authors", "publisher", "ratings_count", "original_publication_date", "genres", "read", "currently_reading", "to_read", "good_book"]]
df_bigml.to_csv(path_or_buf="books-bigml.csv", sep="|", index=False)

## Cargamos los datos en la plataforma BigML

Para ello creamos un nuevo proyecto llamado "Books" en el cual hemos cargado el dataset que hemos confeccionado con los campos mencionados anteriormente.
![carga-csv](images/carga-csv.png)



Como hemos tenido que guardar el CSV con un separador de tuberia, configuramos la fuente de datos de forma adecuada. Es decir, indicandole el tipo de separador de cada registro y el separador de los campos multivalor.
![source-configuration](images/source-configuration.png)



Comprobamos como está interpretando cada uno de los campos para verificar que el tipo de datos interpretado por BigML es el esperado. Podemos observar que el campo _"authors"_ y _"original_publication_date"_ tienen el tipo "text".
![source-configuration](images/dtype-analisis.png)



Queremos modifcarlo. Para ello podemos hacer uso del siguiente menú de configuración:
![source-configuration](images/dtype-settings.png)



Una vez realizados los ajustes, podemos comprobar en la siguiente pantalla que todo es correcto.
![source-configuration](images/dtype-final-analisis.png)


## Creación de un dataset

Una vez tenemos la fuente de datos bien configurada, mediante los botones indicados en la imagen podemos crear el dataset que nos permitirá construir los modelos de decisión (p.ej árboles de exploración de los datos).

![source-configuration](images/create-dataset.png)

### Division del dataset
Partimos en el dataset en otros 2 dataset independientes.

+ Uno con el 80% del total de los datos que lo usaremos para entrenar los modelos
+ y otro con el 20% restante de la información.

Éste último lo usaremos para poder contrastar como de bueno es el modelo construido en las sesiones de entrenamiento.

![source-configuration](images/split-dataset.png)


## Modelo supervisado

### Entrenando el modelo 
Una vez tenemos configurados los datasets llega el momento de entrenar el modelo con uno de ellos. Tal y como hemos comentando anteriormente, usaremos que el contiene el 80% de la información. 

Para ello vamos a crear un modelo supervisado cuyo objetivo sea determinar si un libro será bueno.

![source-configuration](images/create-model.png)

### Resultados
Después de procesar la información, nos devuelve un gráfico forma de árbol con el que podemos experimentar y buscar la rama mas prometedora. En nuestro ha obtenido un nivel de precisión muy elevado. Podemos ver a la derecha las condiciones que ha tenido en cuenta para predecir si se trata de un libro bueno.
 
![tree-prediction](images/tree-prediction.png)


### Evaluación del modelo
Una vez que hemos entrenado el modelo con nuestro dataset de entrenamiento, ha llegado la hora de ponerlo a trabajar contra el otro dataset que habíamos creado reservando el 20% de los datos originales de modo que podamos contrastar cual es la precisión con la que trabaja el modelo y ver la cantidad de falsos positivos que nos da, así como la de falsos negativos.

En la siguiente pantalla podemos configurar los dataset que queremos usar para llevar a cabo la evaluación.

![model-evaluation-setup](images/model-evaluation-setup.png)

Le damos a _"Evaluate"_ y obtenemos los resultados:

![model-evaluation-conclusion](images/model-evaluation-conclusion.png)

Como podemos ver, lo primero que hacemos es establecer el valor _"Positive class"_ del gráfico a _"True"_. Con ello estamos indicando que nos contraste los datos actuales con los de la predicción para saber cuantas veces tenemos un falso positivo (es decir, que indica que un libro va a ser bueno cuando en la realidad no lo es) y cuantas un falso negativo (justo lo contrario, nos indica que el libro va a ser malo para los lectores, pero en realidad será bueno).

Podemos apreciar que tenemos 666 falsos positivos (naranjas en el gŕafico) y 79 falsos negativos (rojos). No es la mejor aproximación que podríamos esperar del modelo, ya que para la industria literaria tener muchos falsos positivos seguramente signifique un gran desembolso económico en marketing y distribución de un producto que no va a tener el éxito esperado. 
Mientras que tener solamente 79 falsos negativos no es tan grave ya que significa que para esos libros no habrá tanta inversión para promover su venta y aún así obtendremos buenas ventas debido a su propia condición de ser bueno en realidad.

Es por ello que es conveniente explorar otros modelos antes de decidir que modelo usar para nuestras predicciones.


## Modelos no supervisados

### Detección de anomalías
En la sección _"Datasets"_  del proyecto podremos seleccionar en un menú contextual de nuestro dataset el nuevo tipo de dataset que queremos generar, por ejemplo, anomalías. Al haberlo seleccionado pasamos a una pantalla que nos permitirá seleccionar sobre que campos queremos explorar las anomalías y construir con ello un dataset nuevo. Por defecto está configurado para detectar 10 anomalías o valores extraños en el dataset, por ejemplo, valoraciones medias negativas de los libros.

Nosotros vamos a configurarlo para que identifique 50 animalías y que genere el dataset libre de ellas.

![anomalies-detection](images/anomalies-detection.png)


Nos aparece la siguiente pantalla en la que se muestra un resumen de las anomalías identificadas para que añadamos las que creamos oportunas antes de crear el dataset. Nosotros hemos seleccionado todas.

![create-50-anomalies-dataset](images/create-50-anomalies-dataset.png)

Si en la imagen anterior no seleccionamos el botón a la izquierda de _"Create dataset"_ nos creará un dataset sólo con las anomalías detectadas. Es sería el resultado del nuevo dataset con solamente las anomalías

![top-50-anomalies-dataset](images/top-50-anomalies-dataset.png)

Sin embargo, lo que nos interesa es justo el caso contrario, de modo que creamos el dataset marcando esa opción. A partir de esos datos es posible crear de nuevo 2 dataset para entrenar otro modelo supervisado y ver si mejoramos respecto al caso anterior.

![training-without-anomalies](images/training-without-anomalies.png)


Entrenamos de nuevo el modelo con el dataset que tiene el 80% de los datos libres de anomalias, y obtenemos el siguiente árbol de exploración
![tree-without-anomalies](images/tree-without-anomalies.png)

Por último pasamos a evaluar que tal rinde en este caso el modelo con la nueva situación para poder contrastarlo con la evaluación anterior y ver si ha mejorado el número de falsos positivos y falsos negativos.

![evaluation-without-anomalies](images/evaluation-without-anomalies.png)

Como podemos observar en la matriz de confusión, el número de falsos positivos y de falsos negativos ha aumentado ligeramente. Podemos concluir por tanto que el segundo modelo entrenado a partir de un dataset libre de anomalias no mejora la predición de si el libro será bueno o no.


## Modelo Random Forest
Otro modo de construir un modelo competente es usando lo que se denomina Random Forest, o Emsemble en BigML. Consiste en entrenar "n" modelos simultáneamente para mejorar la precisión del modelo resultante final.

Para ello vamos a configurar hasta 15 modelos que serán entrenados con el 80% de los datos del dataset original. Y con ello podremos evaluar de nuevo mediante la matriz de confusión si nos ha resultado de ayuda para nuestro conjunto de datos.

![random-forest-config](images/random-forest-config.png)

Podemos comprobar cuales campos tienen más peso en este tipo de modelos

![random-forest-field-importances](images/random-forest-field-importances.png)


Pasamos a evaluar el modelo y a examinar la matriz de confusión en la siguiente imagen.
Podemos comprobar que los resultados no mejoran a los obtenidos anteriormente, ya que pasamos a tener 689 falsos positivos. Sin embargo, el número de falsos negativos baja considerablemente.
Teniendo en cuenta que para el caso que nos ocupa (predecir si es buen libro) debemos minimizar los falsos positivos a costa de aumentar los falsos negativos, nos quedamos con el modelo inicial. 

![evaluate-random-forest](images/evaluate-random-forest.png)
